In [37]:
from STvAR import *
import STvARFileWrite as stfw
import STvARInputs as stvi

In [38]:
DiffOrder = 2
nghostcells = 4
#KOsigma = 0.1  #Kriess-Oliger dissipation.  Irrelevant for linear equations like Klein Gordon

In [39]:
dim = 2
path = "../Source/KleinGordon/"

# RHS

In [40]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")
fileRHS.write(stfw.RHS_Header(dim = 2)) #Standard header for dim = 2 system.

673

In [41]:
stVar.declState = [] #This will hold the names of evolved variables for sympy->AMReX CodeGen

Psi = stVar('Psi', state = True)  #Setting state = true means it will be treated as an evolved variable and added to the list stVar.declState
Pi = stVar('Pi', state = True)

fileRHS.write(Psi.AMReXSymb2State())  #This will define a symbol Psi initialized to the indexed multifab expression
fileRHS.write(Pi.AMReXSymb2State())

55

In [42]:
#This will hold the list of evolved variables for the plot files.

fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write(stfw.VarString(stVar.declState))
fileVARS.close()

In [43]:
#DstVar operates on stVar objects and returns finite differenced indexed expressions.
#AMReXSymb2Expr() writes out an executable line defining a symbol initialized to the expression contained in the stVar object.
#Try replacing fileRHS.write with print to see what they look like.

fileRHS.write(DstVar(Psi, 1, orderD = DiffOrder).AMReXSymb2Expr()) 
fileRHS.write(DstVar(Psi, 2, orderD = DiffOrder).AMReXSymb2Expr())
#fileRHS.write(DstVar(Psi, 'KO', orderD = DiffOrder, sigma = KOsigma).AMReXSymb2Expr())

1881

In [44]:
m = stVar('m')  #stVar objects can be used for constants as well.  Here we define a symbol for the mass.
m.expr = 1  #Mass set to 1, by setting the expression (expr) to 1.
fileRHS.write(m.AMReXSymb2Expr())

28

In [45]:
RHS_Psi = stVar('Psi')  #Defining a separate stVar object holding the RHS expression for Psi.  It has the same symbolic name as the original stVar object.

RHS_Psi.expr = Pi.symb  #Expression (expr) of RHS_Psi set to Pi.

# Note: .symb refers to the symbol holding information.  It isn't really necessary, but makes code look more clean.
# You could for example instead write RHS_Psi.expr = Pi.expr, in which case you'll get the rhs equal to the indexed version.
# Again, try printing the statements to get an idea of how they are organized.

In [46]:
RHS_Pi = stVar('Pi')

for i in range(dim):
    RHS_Pi.expr += Dsymb(Psi.symb, str(i)+str(i))

RHS_Pi.expr += -m.symb**2*Psi.symb

In [47]:
#Writing RHS expressions to the RHS file.

fileRHS.write(RHS_Psi.AMReXSetRHS())
fileRHS.write(RHS_Pi.AMReXSetRHS())

78

In [48]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()

# Diagnostics file

You can use these to plot any diagnostic functions that are not directly evolved by the equations of motion.
E.g. you. could plot the energy density as a function of psi and its derivatives.

In [13]:
fileDIAG = open(path+"ET_Integration_Diag_K.H", "w+")
fileDIAG.write(stfw.Diag_Header(dim = 2))

684

In [14]:
#We will again need symbols for Pi and Psi here.

fileDIAG.write(Psi.AMReXSymb2State())
fileDIAG.write(Pi.AMReXSymb2State())

55

In [15]:
#We only need first order spatial derivatives for Psi

fileDIAG.write(DstVar(Psi, 1, orderD = DiffOrder).AMReXSymb2Expr()) 

639

In [16]:
fileDIAG.write(m.AMReXSymb2Expr())

28

In [17]:
#Now define the expression for energy density.  First we declare a diagnostic variable

stVar.declDiag = []

EnDens = stVar('EnDens', diag = True)

In [18]:
fileDIAGVARS = open(path+"ET_Integration_Diagnostic_Variables.H","w+")
fileDIAGVARS.write(stfw.VarString(stVar.declDiag))
fileDIAGVARS.close()

In [19]:
EnDens.expr = 1/2*(Pi.symb)**2 + 1/2*(m.symb**2)*(Psi.symb**2)+1/2*Dsymb(Psi.symb, '0')**2 + 1/2*Dsymb(Psi.symb, '1')**2

In [20]:
fileDIAG.write(EnDens.AMReXSetDiag())

148

In [21]:
stVar.declDiag

['EnDens']

In [22]:
fileDIAG.write(stfw.Closer())
fileDIAG.close()

# Problem Setup file

Defines the variables and diagnostics to be tracked, as well as ghost cells, and namespaces for the components.

In [23]:
fileSETUP = open(path+"ET_Integration_Setup_K.H", "w+")
fileSETUP.write(stfw.Write_Setup_File(stVar.declState, stVar.declInit, stVar.declDiag, nghostcells))
fileSETUP.close()

# Post updating

We can do any time step post updating here. For example we could insist on a particular gauge condition here. It would be written in the same format as RHS and DIAG files.  For Klein Gordon it is blank since it is not used.

In [24]:
filePOST = open(path+"ET_Integration_Post_Update_K.H", "w+")
filePOST.write(stfw.PostUpdate_Header())
filePOST.write(stfw.Closer())
filePOST.close()

# Initializations of evolved variables

Here is where we will define the initial conditions for the KG fields.

In [25]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")
fileINIT.write(stfw.InitFromScratch_Header())

612

In [26]:
PsiInit = stVar('Psi')
PiInit = stVar('Pi')

In [27]:
x = stVarRank1('x') #Need a coordinate symbol to define initial condition

In [28]:
PsiInit.expr = sp.exp(-(x.symb[0]-5)**2 - x.symb[1]**2)+sp.exp(-(x.symb[0]+5)**2 - x.symb[1]**2)
PiInit.expr = 0

In [29]:
fileINIT.write(PsiInit.AMReXSetState())
fileINIT.write(PiInit.AMReXSetState())

42

In [30]:
fileINIT.write(stfw.Closer())
fileINIT.close()

# AMR tagging conditions

We will defining the cell tagging conditions here.  Virtually any function of the field variables and their derivatives can serve as a tagging condition.  Here we will simply use |Psi| > threshold as the condition.

In [31]:
fileAMRTAG = open(path+"ET_Integration_AMR_Cell_Tag_K.H", "w+")
fileAMRTAG.write(stfw.AMRtagging_Header())

744

In [32]:
fileAMRTAG.write(Psi.AMReXSymb2State())

57

In [33]:
AbsPsi = stVar('AbsPsi')
AbsPsi.expr = sp.Abs(Psi.symb)

In [34]:
fileAMRTAG.write(AbsPsi.AMReXSymb2Expr())

46

In [35]:
fileAMRTAG.write(TagCondition(AbsPsi, '>','symb'))

41

In [36]:
fileAMRTAG.write(stfw.Closer())
fileAMRTAG.close()